In [10]:
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torchvision import datasets, transforms
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

epslions = [0, .5, .1, .15, .2, .25, .3, .35, .4]
pretrained_model = "data/lenet_mnist_model.pth"
use_cuda = True

class Net(nn.Module):
    def __init__(self) -> None:
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = nn.Conv2d(10,20,kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
    
    def forward(self,x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x = x.view(-1,320)
        x = F.relu(self.fc1(x));
        x = F.dropout(x,training = self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',train = False,download = True,transform = transforms.Compose([
        transforms.ToTensor(),
        ])),
    batch_size = 1,shuffle = True
)

print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

model = Net().to(device)
model.load_state_dict(torch.load(pretrained_model, map_location = 'cpu'))
model.eval()

def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# def test(model, device, test_loader):
#     correct = 0;
    
#     for data, target in test_loader:
#         data, target = data.to(device), target.to(device)
#         data.requires_grad = True
#         output = model(data)
#         pred_result = output.max(1,keepdim = True)[1]
#         if pred_result.item() == target.item():
#             correct += 1
    
#     print("correct / total : {} / {}".format(correct,len(test_loader)))
#     return correct, len(test_loader)


# test(model,device,test_loader)

def demo(model, device, img):  
    output = model(img)
    
img_path = "D:\\Env\\vscode\\DeepLearning\\FGSM\\dataset\\test\\dog.jpg"
img = Image.open(img_path)
demo(model,device,img)
    
    


CUDA Available:  False


TypeError: conv2d() received an invalid combination of arguments - got (JpegImageFile, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!JpegImageFile!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!JpegImageFile!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)


In [9]:
from PIL import Image

img_path = "D:\\Env\\vscode\\DeepLearning\\FGSM\\dataset\\test\\dog.jpg"
img = Image.open(img_path)
img.show()